In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from cipher import *
from scheme import *

In [3]:
myring = Ring(seed=1234)
parms = Parameters(logp = 30, logq = 150, logn = 12)
nslots = 2**parms.logn
context = Context(parms, myring)

sk = context.generate_secret_key()

print(context.params)

 logp: 30
 logq: 150 
 logn: 12


In [4]:
keys = {"mult":context.generate_mult_key(),
        "rot":{'1':'hi1',
               '2':'hi2',
               '4':'hi4',
               '8':'hi8'}}
ev = Evaluator(keys) # Evaluator도 그냥 context만 넣게 할까? 
encryptor = Encryptor(context)

decryptor = Decryptor(sk)

ctxt = encryptor.encrypt(np.array([1,2,3,4,5,6,7,8]))

In [5]:
ctxt._enckey_hash

2797295717590756220

In [12]:
def fhe_sum(ctxt, evaluator): 
    arr = ctxt._arr
    n_elements = ctxt._n_elements
    log2n = np.log2(n_elements).astype(int)

    for i in range(log2n):
        tmp = evaluator.copy(ctxt)
        evaluator.lrot(tmp, 2**i, inplace=True)
        evaluator.add(ctxt, tmp, inplace=True)
    
    return ctxt

In [15]:
summed = fhe_sum(ctxt, ev)

In [59]:
np.sum(arr)

36

In [11]:
def my_var(arrx):
    n = arrx.__len__()
    sumx = np.sum(arrx)
    meanx = sumx / n
    sub = arrx - meanx
    squared = sub * sub
    return np.sum(squared) * (1./(n))
    

In [12]:
arrx = np.array([1,2,3,4])

In [13]:
diff = np.sum(arrx) - np.mean(arrx)

In [39]:
1.5**2 + 0.5**2 + 0.5**2 + 1.5**2

5.0

In [28]:
np.sum(diff**2)

56.25

In [14]:
my_var(arrx)

1.25

In [15]:
np.mean(arrx)

2.5

In [16]:
np.var(arrx)

1.25

In [ ]:
def my_cov(arrx, arry):
    sumx = np.sum(arrx)
    sumy = np.sum(arry)
    meanx = sumx / len(arrx)
    meany = sumy / len(arry)
    
    

In [4]:
np.log2(5e7)

25.575424759098897